In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import nltk.classify.util
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.classify import NaiveBayesClassifier
import numpy as np
import re
import string
import nltk
%matplotlib inline

In [7]:
raw_reviews = pd.read_csv(r"amazon_reviews.csv")
raw_reviews.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,0528881469,"[0, 0]",5.0,We got this GPS for my husband who is an (OTR)...,"06 2, 2013",AO94DHGC771SJ,amazdnu,Gotta have GPS!,1.370131e+09
1,1,0528881469,"[12, 15]",1.0,"I'm a professional OTR truck driver, and I bou...","11 25, 2010",AMO214LNFCEI4,Amazon Customer,Very Disappointed,1.290643e+09
2,2,0528881469,"[43, 45]",3.0,"Well, what can I say. I've had this unit in m...","09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1st impression,1.283990e+09
3,3,0528881469,"[9, 10]",2.0,"Not going to write a long review, even thought...","11 24, 2010",A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""","Great grafics, POOR GPS",1.290557e+09
4,4,0528881469,"[0, 0]",1.0,I've had mine for a year and here's what we go...,"09 29, 2011",A24EV6RXELQZ63,Wayne Smith,"Major issues, only excuses for support",1.317254e+09


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      999 non-null    int64  
 1   asin            999 non-null    object 
 2   helpful         999 non-null    object 
 3   overall         999 non-null    float64
 4   reviewText      997 non-null    object 
 5   reviewTime      999 non-null    object 
 6   reviewerID      999 non-null    object 
 7   reviewerName    994 non-null    object 
 8   summary         999 non-null    object 
 9   unixReviewTime  999 non-null    float64
dtypes: float64(2), int64(1), object(7)
memory usage: 78.2+ KB


In [9]:
process_reviews=raw_reviews.copy()

#Checking for null values
process_reviews.isnull().sum()

Unnamed: 0        0
asin              0
helpful           0
overall           0
reviewText        2
reviewTime        0
reviewerID        0
reviewerName      5
summary           0
unixReviewTime    0
dtype: int64

In [12]:
process_reviews['reviewText']=process_reviews['reviewText'].fillna('Missing')

In [13]:
process_reviews['reviews']=process_reviews['reviewText']+process_reviews['summary']
process_reviews=process_reviews.drop(['reviewText', 'summary'], axis=1)
process_reviews.head()

,Unnamed: 0,asin,helpful,overall,reviewTime,reviewerID,reviewerName,unixReviewTime,reviews
0,0,0528881469,"[0, 0]",5.0,"06 2, 2013",AO94DHGC771SJ,amazdnu,1.370131e+09,We got this GPS for my husband who is an (OTR)...
1,1,0528881469,"[12, 15]",1.0,"11 25, 2010",AMO214LNFCEI4,Amazon Customer,1.290643e+09,"I'm a professional OTR truck driver, and I bou..."
2,2,0528881469,"[43, 45]",3.0,"09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1.283990e+09,"Well, what can I say. I've had this unit in m..."
3,3,0528881469,"[9, 10]",2.0,"11 24, 2010",A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""",1.290557e+09,"Not going to write a long review, even thought..."
4,4,0528881469,"[0, 0]",1.0,"09 29, 2011",A24EV6RXELQZ63,Wayne Smith,1.317254e+09,I've had mine for a year and here's what we go...


In [14]:
process_reviews['overall'].value_counts()

overall
5.0    582
4.0    188
3.0     94
1.0     87
2.0     48
Name: count, dtype: int64

In [15]:
def f(row):
    
    
    if row['overall'] == 3.0:
        val = 'Neutral'
    elif row['overall'] == 1.0 or row['overall'] == 2.0:
        val = 'Negative'
    elif row['overall'] == 4.0 or row['overall'] == 5.0:
        val = 'Positive'
    else:
        val = -1
    return val

In [16]:
process_reviews['sentiment'] = process_reviews.apply(f, axis=1)
process_reviews.head()

,Unnamed: 0,asin,helpful,overall,reviewTime,reviewerID,reviewerName,unixReviewTime,reviews,sentiment
0,0,0528881469,"[0, 0]",5.0,"06 2, 2013",AO94DHGC771SJ,amazdnu,1.370131e+09,We got this GPS for my husband who is an (OTR)...,Positive
1,1,0528881469,"[12, 15]",1.0,"11 25, 2010",AMO214LNFCEI4,Amazon Customer,1.290643e+09,"I'm a professional OTR truck driver, and I bou...",Negative
2,2,0528881469,"[43, 45]",3.0,"09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1.283990e+09,"Well, what can I say. I've had this unit in m...",Neutral
3,3,0528881469,"[9, 10]",2.0,"11 24, 2010",A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""",1.290557e+09,"Not going to write a long review, even thought...",Negative
4,4,0528881469,"[0, 0]",1.0,"09 29, 2011",A24EV6RXELQZ63,Wayne Smith,1.317254e+09,I've had mine for a year and here's what we go...,Negative


In [17]:
process_reviews['sentiment'].value_counts()

sentiment
Positive    770
Negative    135
Neutral      94
Name: count, dtype: int64

In [18]:
# new data frame which has date and year
new = process_reviews["reviewTime"].str.split(",", n = 1, expand = True) 
  
# making separate date column from new data frame 
process_reviews["date"]= new[0] 
  
# making separate year column from new data frame 
process_reviews["year"]= new[1] 

process_reviews=process_reviews.drop(['reviewTime'], axis=1)
process_reviews.head()

,Unnamed: 0,asin,helpful,overall,reviewerID,reviewerName,unixReviewTime,reviews,sentiment,date,year
0,0,0528881469,"[0, 0]",5.0,AO94DHGC771SJ,amazdnu,1.370131e+09,We got this GPS for my husband who is an (OTR)...,Positive,06 2,2013
1,1,0528881469,"[12, 15]",1.0,AMO214LNFCEI4,Amazon Customer,1.290643e+09,"I'm a professional OTR truck driver, and I bou...",Negative,11 25,2010
2,2,0528881469,"[43, 45]",3.0,A3N7T0DY83Y4IG,C. A. Freeman,1.283990e+09,"Well, what can I say. I've had this unit in m...",Neutral,09 9,2010
3,3,0528881469,"[9, 10]",2.0,A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""",1.290557e+09,"Not going to write a long review, even thought...",Negative,11 24,2010
4,4,0528881469,"[0, 0]",1.0,A24EV6RXELQZ63,Wayne Smith,1.317254e+09,I've had mine for a year and here's what we go...,Negative,09 29,2011


In [20]:
new1 = process_reviews["date"].str.split(" ", n = 1, expand = True) 

# adding month to the main dataset 
process_reviews["month"]= new1[0] 
  
# adding day to the main dataset 
process_reviews["day"]= new1[1] 

process_reviews=process_reviews.drop(['date'], axis=1)
process_reviews.head()

,Unnamed: 0,asin,helpful,overall,reviewerID,reviewerName,unixReviewTime,reviews,sentiment,year,month,day
0,0,0528881469,"[0, 0]",5.0,AO94DHGC771SJ,amazdnu,1.370131e+09,We got this GPS for my husband who is an (OTR)...,Positive,2013,06,2
1,1,0528881469,"[12, 15]",1.0,AMO214LNFCEI4,Amazon Customer,1.290643e+09,"I'm a professional OTR truck driver, and I bou...",Negative,2010,11,25
2,2,0528881469,"[43, 45]",3.0,A3N7T0DY83Y4IG,C. A. Freeman,1.283990e+09,"Well, what can I say. I've had this unit in m...",Neutral,2010,09,9
3,3,0528881469,"[9, 10]",2.0,A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""",1.290557e+09,"Not going to write a long review, even thought...",Negative,2010,11,24
4,4,0528881469,"[0, 0]",1.0,A24EV6RXELQZ63,Wayne Smith,1.317254e+09,I've had mine for a year and here's what we go...,Negative,2011,09,29


In [21]:
process_reviews.head()

,Unnamed: 0,asin,helpful,overall,reviewerID,reviewerName,unixReviewTime,reviews,sentiment,year,month,day
0,0,0528881469,"[0, 0]",5.0,AO94DHGC771SJ,amazdnu,1.370131e+09,We got this GPS for my husband who is an (OTR)...,Positive,2013,06,2
1,1,0528881469,"[12, 15]",1.0,AMO214LNFCEI4,Amazon Customer,1.290643e+09,"I'm a professional OTR truck driver, and I bou...",Negative,2010,11,25
2,2,0528881469,"[43, 45]",3.0,A3N7T0DY83Y4IG,C. A. Freeman,1.283990e+09,"Well, what can I say. I've had this unit in m...",Neutral,2010,09,9
3,3,0528881469,"[9, 10]",2.0,A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""",1.290557e+09,"Not going to write a long review, even thought...",Negative,2010,11,24
4,4,0528881469,"[0, 0]",1.0,A24EV6RXELQZ63,Wayne Smith,1.317254e+09,I've had mine for a year and here's what we go...,Negative,2011,09,29
